# Fabriano


**Fabriano** : Based on Eakins.

In [1]:
import tensorflow as tf
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import time
import sys
import io

from sklearn.model_selection import train_test_split

from nn_helpers import *

%config InlineBackend.figure_format = 'retina'
darkness = True

/Users/andrew/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Log number : 899574

In [2]:
# log_random_serial_number = '{}'.format(np.random.randint(100000,999999))
# print(log_random_serial_number)

In [3]:
log_direc = log_dir('899574')
print(log_direc)

../../../logs/899574


In [4]:
flags = tf.app.flags

# flag 'f' prevents an error due to using flags in jupyter notebook
#https://github.com/tensorflow/tensorflow/issues/17702#issuecomment-387335646
flags.DEFINE_string('f', '', 'kernel')

# Data parameters
flags.DEFINE_float("testing_data_percentage", 0.20,"Percentage of the data to use for testing. (default: 0.10)")
flags.DEFINE_string('data_path', '../../../data/wrapped_20000_gaussian_spikes_100_medium_noise_rand_B.bz2', "Data source for training and testing")

# training parameters
flags.DEFINE_integer('num_epochs', 1000, "Number of training epochs")
flags.DEFINE_integer('num_batches', 500, "Number of batches per epoch")
flags.DEFINE_float('keep_prob', 0.50, "Dropout probability")
flags.DEFINE_float('learning_rate', 0.00001, "Learning rate")

# saving parameters
flags.DEFINE_string('log_directory', log_direc, "Log save directory")
flags.DEFINE_integer('image_save_interval', 5, "Epoch to save images on.")

FLAGS = tf.flags.FLAGS

In [5]:
data = load_data(FLAGS.data_path)

In [6]:
scaled_input, scaled_output, input_scaler, output_scaler = scale_data(data)
X_train, X_test, y_train, y_test = train_test_split(scaled_input, scaled_output,
                                                    test_size = FLAGS.testing_data_percentage,
                                                    random_state = np.random.seed(int(time.time())))

In [7]:
number_of_inputs  = scaled_input.shape[1]
number_of_outputs = scaled_output.shape[1]

# these should probably become flags as well
layer_1_nodes = 256
layer_2_nodes = 128
layer_3_nodes = 64

In [8]:
tf.reset_default_graph()

In [9]:
def gen_plot(predicted_values, actual_values):
    """Create a pyplot plot and save to buffer."""
    prediction_unscaled = output_scaler.inverse_transform(predicted_values)
    actual_unscaled = output_scaler.inverse_transform(actual_values)

    sorting_idx = np.argsort(actual_unscaled.T[0])
        
    fig, ax = plt.subplots(figsize = (5, 3), dpi = 144)

    ax.plot(prediction_unscaled.T[0][sorting_idx], linestyle = 'none', marker = '.', markersize = 1, color = 'darkblue')
    ax.plot(actual_unscaled.T[0][sorting_idx],     linestyle = 'none', marker = '.', markersize = 1, color = '#E50000', alpha = 0.50, )        
#     ax.set_ylim(-200,200)
    # ylims changed on run C
    ax.set_ylim(-250,250)


    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi = 144)
    plt.close(fig)
    buf.seek(0)

    return buf.getvalue()

In [10]:
with tf.variable_scope('input_X'):
    X  = tf.placeholder(tf.float32, shape = (None, number_of_inputs))
    
with tf.variable_scope('input_y'):
    y  = tf.placeholder(tf.float32, shape = (None, 1))
    
with tf.variable_scope('keep'):
    keep_prob = tf.placeholder(tf.float32)
    
with tf.variable_scope('layer_1'):

    biases = tf.get_variable(name = "biases1",
                             shape = [layer_1_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights1",
                              shape  = [number_of_inputs, layer_1_nodes], 
                              initializer = tf.contrib.layers.xavier_initializer())


    layer_1_output = tf.nn.leaky_relu(tf.matmul(X, weights) + biases)

with tf.variable_scope('dropout_12'):
    drop_out_12 = tf.nn.dropout(layer_1_output, keep_prob)     
      
with tf.variable_scope('layer_2'):

    biases = tf.get_variable(name = "biases2",
                             shape = [layer_2_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights2",
                              shape  = [layer_1_nodes, layer_2_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_2_output = tf.nn.relu(tf.matmul(drop_out_12, weights) + biases)
    
with tf.variable_scope('layer_3'):

    biases = tf.get_variable(name = "biases2",
                             shape = [layer_3_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights2",
                              shape  = [layer_2_nodes, layer_3_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)
    
with tf.variable_scope('output'):

    biases = tf.get_variable(name = "biases_out",
                             shape = [number_of_outputs],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights_out",
                              shape  = [layer_3_nodes, number_of_outputs],
                         initializer = tf.contrib.layers.xavier_initializer())


    prediction = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)
    
with tf.variable_scope('cost'):

    cost = tf.sqrt(tf.reduce_mean(tf.squared_difference(prediction, y)))

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate, epsilon=1e-08).minimize(cost)

with tf.variable_scope('image'):
    image_buf  = tf.placeholder(tf.string, shape=[])
    epoch_image = tf.expand_dims(tf.image.decode_png(image_buf, channels=4), 0)
    
with tf.variable_scope('logging'):
    
    tf.summary.image('prediction_vs_actual', epoch_image)
    tf.summary.histogram('predictions', prediction)
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

In [11]:
saver = tf.train.Saver()

In [12]:
with tf.Session() as session:

#     session.run(tf.global_variables_initializer())
    saver.restore(session,'../../../logs/652719B/trained_model.ckpt-949')
    training_writer = tf.summary.FileWriter(log_direc + '/training', session.graph)
    testing_writer = tf.summary.FileWriter(log_direc + '/testing', session.graph)
    model_save_location = log_direc + '/trained_model.ckpt'

    
    for epoch in range(FLAGS.num_epochs):
        
        split_permutation = np.random.permutation(X_train.shape[0])
        X_train_batches = np.vsplit(X_train[split_permutation], FLAGS.num_batches)
        y_train_batches = np.vsplit(y_train[split_permutation], FLAGS.num_batches)
        
        for i in range(FLAGS.num_batches):
            X_train_batch = X_train_batches[i]
            y_train_batch = y_train_batches[i]
        
            session.run(optimizer,
                        feed_dict = {X: X_train_batch, y: y_train_batch, keep_prob : 0.50})

#         if (epoch + 1) % FLAGS.image_save_interval == 0:
            
#             saver.save(session, model_save_location, epoch)
        prediction_scaled_test = session.run(prediction, feed_dict = {X: X_train, keep_prob : 1.00})
        image_buffer = gen_plot(prediction_scaled_test, y_train)
#             print('xx')
#             _ = session.run(epoch_image, feed_dict = {image_buf: image_buffer})
            
        training_feed = {X: X_train, y: y_train, keep_prob : 1.00,image_buf: image_buffer}
        training_cost, training_summary = session.run([cost, summary], feed_dict = training_feed)

        prediction_scaled_test = session.run(prediction, feed_dict = {X: X_test, keep_prob : 1.00})
        image_buffer = gen_plot(prediction_scaled_test, y_test)
        
        testing_feed =  {X: X_test, y: y_test, keep_prob : 1.00,image_buf: image_buffer}
        testing_cost, testing_summary = session.run([cost, summary],feed_dict = testing_feed)
        
        sys.stdout.write('\r epoch: {:4.0f} testing_cost: {:2.10f}'.format(epoch, testing_cost))
        
        training_writer.add_summary(training_summary, epoch)
        testing_writer.add_summary(testing_summary, epoch)
        testing_writer.flush()
        training_writer.flush()

        if (epoch + 1) % 50 == 0:
            
            saver.save(session, model_save_location, epoch)



            
#             prediction_unscaled = output_scaler.inverse_transform(prediction_scaled)
#             actual_unscaled = output_scaler.inverse_transform(y_test)
            
#             sorting_idx = np.argsort(actual_unscaled.T[0])
#             fig, ax = plt.subplots(figsize = (8.5, 3), dpi = 100)
            
#             ax.plot(prediction_unscaled.T[0][sorting_idx], linestyle = 'none', marker = '.', markersize = 1, color = 'darkblue')
#             ax.plot(actual_unscaled.T[0][sorting_idx],     linestyle = 'none', marker = '.', markersize = 1, color = '#E50000', alpha = 0.50, )
#             plt.show()
            
    saver.save(session, model_save_location, epoch)

INFO:tensorflow:Restoring parameters from ../../../logs/652719B/trained_model.ckpt-949
 epoch:  601 testing_cost: 0.0776392594

KeyboardInterrupt: 